In [ ]:
print('ok')

In [ ]:
%pwd

In [ ]:
import os
os.chdir('../')


In [ ]:
%pwd

In [ ]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter



In [ ]:
def load_pdf_files(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents





In [ ]:
extracted_data = load_pdf_files(data= "Data/")


In [ ]:
extracted_data

In [ ]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks


In [ ]:
text_chunks = text_split(extracted_data)
print("length of text chunks: ", len(text_chunks))

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings


In [ ]:
#download the embeddings from hugging face

def download_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [ ]:
embeddings = download_embeddings()


In [ ]:
query_result = embeddings.embed_query("Hello world")
print("length", len(query_result))

In [ ]:
#query_result

In [ ]:
from dotenv import load_dotenv
load_dotenv()


In [ ]:
import os

PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")

In [ ]:
import pinecone
from pinecone import ServerlessSpec
import os

pc = pinecone.Pinecone(api_key=PINECONE_API_KEY)

index_name = 'medicalchatbot'

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [ ]:
import os
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")

In [ ]:
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings
import pinecone

# Initialize Pinecone client
pc = pinecone.Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

In [ ]:
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"),
    index_name=index_name,
    namespace="medical-namespace"  # Optional but recommended
)

In [ ]:
#load existing index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    embedding=embeddings,
    index_name=index_name,
    namespace="medical-namespace"
)





In [ ]:
docsearch

In [ ]:
retriever = docsearch.as_retriever(search_type="similarity",   search_kwargs={"k": 3})


In [ ]:
retriever.get_relevant_documents("What is the cause of the Acne?")

In [ ]:
retrieved_docs = retriever.invoke("What is acne?")
retrieved_docs


In [ ]:
import requests
import os
from langchain_core.runnables import RunnableLambda

# Configure API key
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY  # Replace with your key
API_KEY = os.getenv("GEMINI_API_KEY")

# Direct API integration for Gemini 2.0 Flash
def gemini_flash(prompt_text: str) -> str:
    url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?key={API_KEY}"
    headers = {'Content-Type': 'application/json'}
    payload = {
        "contents": [{
            "parts": [{"text": prompt_text}]
        }]
    }
    
    try:
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()
        return response.json()['candidates'][0]['content']['parts'][0]['text']
    except Exception as e:
        return f"API Error: {str(e)}"

# Medical RAG chain
template = """As a medical assistant, answer using this context:
{context}

Question: {question}

Format:
- Summary (2 sentences)
- 3 Key Symptoms
- Immediate Actions"""

prompt = ChatPromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt
    | RunnableLambda(lambda x: gemini_flash(x.messages[0].content))
    | StrOutputParser()
)

# Test with rate limiting
from tenacity import retry, wait_exponential

@retry(wait=wait_exponential(multiplier=1, min=4, max=10))
def safe_invoke(question):
    return rag_chain.invoke(question)

response = safe_invoke("What are the warning signs of a stroke?")
print(response)

In [ ]:
response = safe_invoke("What is Acne?")
print(response)

In [ ]:
%pwd